In [1]:
%matplotlib inline

In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, r2_score, mean_squared_error, confusion_matrix

In [3]:
thing = pd.read_csv("GLC03122015.csv")

In [4]:
def analyze(dataset, cell, details = True):
    values = dataset[cell].unique()
    
    print(len(values))
    print(values)
    if(details):
        for value in values:
            print(value, ' -> ', len(dataset[dataset[cell] == value]))

In [5]:
#Useful
analyze(thing, "date_", False)

2148
['03/07/2011 08:00:00 AM +0000' '04/01/2008 07:00:00 AM +0000'
 '01/30/2015 08:00:00 AM +0000' ... '12/22/2009 08:00:00 AM +0000'
 '09/27/2007 07:00:00 AM +0000' '11/18/2007 08:00:00 AM +0000']


In [6]:
#Useless
analyze(thing, "time_")

3
['12/30/1899 08:00:00 AM +0000' nan '01/12/1900 08:00:00 AM +0000']
12/30/1899 08:00:00 AM +0000  ->  1268
nan  ->  0
01/12/1900 08:00:00 AM +0000  ->  1


In [7]:
analyze(thing, "hazard_typ")

2
['landslide' 'Lanslide']
landslide  ->  6787
Lanslide  ->  1


In [8]:
analyze(thing, "landslide_", False)
# TODO: Fix Landslide & landslide     rename - landslide_type

15
['Mudslide' 'Landslide' 'Complex' 'Other' 'Rock_Fall' 'Debris_Flow'
 'Rockfall' 'Snow_Avalanche' 'Unknown' 'Translational_Slide' 'mudslide'
 'Creep' 'Lahar' 'Riverbank_Collapse' 'landslide']


In [9]:
# too little to give a fuck
analyze(thing, "storm_name", True)

165
[nan 'Trami' 'Cyclone Giri' 'Hurricane Felix' '3 days of rain'
 'Cyclone Nargis' 'Hudhud' 'Typhoon Kompasu' 'and Rain'
 'Tropical Storm Richard' 'Wipha' 'Tropical Cyclone Hubert'
 'Typhoon Morakot' 'Hurricane Tomas' 'Manuel' 'Lusi' 'Kong Rey'
 'Typhoon Ketsana' 'Utor' 'Tropical Storm Olga' 'Typhoon Sinlaku'
 'Typhoon Fanapi' 'Storm Xynthia' 'Luis' 'Typhoon Sepat' 'Typhoon Megi'
 'Soulik' 'Hurricane Karl and Tropical Storm Matthew'
 'Supertyphoon Juan (Megi)' 'Typhoon Chedeng (Songda)' 'Rammasun'
 'Hurricane Beatriz' 'Santi' 'Tropical Storm Meranti'
 'Tropical Storm Tomas' 'Mangkhut' 'Tropical Storm Dodong (Sarika)'
 'Hurricane Dora' 'Cyclone Wilma' 'Tropical Storm Alma'
 'Tropical Depression Parma' 'Typhoon Ondoy (Ketsana)' 'Neoguri (Florita)'
 'Hurricane Frank' 'Haiyan' 'Tropical Cyclone Wilma' 'Toraji'
 'Tropical Cyclone Maring' 'Typhoon No. 2 and March 11th earthquake'
 'Usagi' 'Agaton' 'Hurricane Sandy' '1 hour of rain'
 'Tropical Storm Kammuri' 'Hurricane Dean' 'Typhoon Chanth

In [10]:
analyze(thing, "landslide1", True)
#TODO: filter & rename - severity

10
['Medium' 'Small' 'Large' 'Very_large' 'unknown' 'large' 'medium' 'small'
 'Extra Large' nan]
Medium  ->  4851
Small  ->  1277
Large  ->  482
Very_large  ->  79
unknown  ->  26
large  ->  23
medium  ->  33
small  ->  14
Extra Large  ->  1
nan  ->  0


In [11]:
#BS
analyze(thing, "cat_src", True)

3
['glc' 'test' nan]
glc  ->  6695
test  ->  92
nan  ->  0


In [12]:
thing = thing.drop(['the_geom', "hazard_typ", 'OBJECTID', 'id', 'time_', "storm_name", 'source_nam', 'fatalities', 'injuries', 'source_lin', 'location_a', 'photos_lin', 'cat_src', 'cat_id', 'countrynam', 'adminname1', 'adminname2', 'population', 'countrycod', 'continentc', 'key_', 'version', 'user_id', 'tstamp', 'changeset_', 'distance', 'near', 'nearest_pl'], axis = 1)

In [13]:
thing.rename(columns={'landslide1':'severity', 'landslide_':'landslide_type', 'date_':'date'}, inplace=True)

In [14]:
thing.head()

,date,country,landslide_type,trigger,severity,latitude,longitude
0,03/07/2011 08:00:00 AM +0000,United States,Mudslide,Downpour,Medium,41.5585,-73.4020
1,04/01/2008 07:00:00 AM +0000,Indonesia,Landslide,Rain,Medium,0.1115,113.9171
2,01/30/2015 08:00:00 AM +0000,NaN,Mudslide,Rain,Medium,52.3545,-127.6980
3,09/24/2010 07:00:00 AM +0000,Canada,Complex,Downpour,Medium,50.7053,-127.5062
4,08/31/2014 07:00:00 AM +0000,NaN,Mudslide,Downpour,Small,53.3319,-132.4149


In [15]:
analyze(thing, "severity", True)

10
['Medium' 'Small' 'Large' 'Very_large' 'unknown' 'large' 'medium' 'small'
 'Extra Large' nan]
Medium  ->  4851
Small  ->  1277
Large  ->  482
Very_large  ->  79
unknown  ->  26
large  ->  23
medium  ->  33
small  ->  14
Extra Large  ->  1
nan  ->  0


In [16]:
#analyze(thing, "trigger", True)

In [17]:
#TODO: fix duplicates 'small' 'Small'
#TODO: deal with date(timestamps) - Scaler
#TODO: fix missing values

In [18]:
thing['severity'] = thing['severity'].str.lower().replace('unknown', np.nan)
thing['trigger'] = thing['trigger'].str.lower().replace('unknown', np.nan)
thing['landslide_type'] = thing['landslide_type'].str.lower().replace('unknown', np.nan)
thing['country'] = thing['country'].str.lower().replace('unitedâ\xa0states', 'united states')
thing.head()

,date,country,landslide_type,trigger,severity,latitude,longitude
0,03/07/2011 08:00:00 AM +0000,united states,mudslide,downpour,medium,41.5585,-73.4020
1,04/01/2008 07:00:00 AM +0000,indonesia,landslide,rain,medium,0.1115,113.9171
2,01/30/2015 08:00:00 AM +0000,NaN,mudslide,rain,medium,52.3545,-127.6980
3,09/24/2010 07:00:00 AM +0000,canada,complex,downpour,medium,50.7053,-127.5062
4,08/31/2014 07:00:00 AM +0000,NaN,mudslide,downpour,small,53.3319,-132.4149


## we need to cure the unknown

In [19]:
analyze(thing, "severity", True)

6
['medium' 'small' 'large' 'very_large' nan 'extra large']
medium  ->  4884
small  ->  1291
large  ->  505
very_large  ->  79
nan  ->  0
extra large  ->  1


In [20]:
analyze(thing, "trigger", True)

16
['downpour' 'rain' 'snowfall_snowmelt' nan 'earthquake'
 'no_apparent_trigger' 'freeze_thaw' 'continuous_rain' 'construction'
 'monsoon' 'tropical_cyclone' 'mining_digging' 'dam_embankment_collapse'
 'other' 'snowfall' 'flooding']
downpour  ->  3890
rain  ->  1575
snowfall_snowmelt  ->  45
nan  ->  0
earthquake  ->  34
no_apparent_trigger  ->  4
freeze_thaw  ->  13
continuous_rain  ->  271
construction  ->  22
monsoon  ->  70
tropical_cyclone  ->  457
mining_digging  ->  38
dam_embankment_collapse  ->  4
other  ->  12
snowfall  ->  1
flooding  ->  14


In [21]:
analyze(thing, "landslide_type", True)

13
['mudslide' 'landslide' 'complex' 'other' 'rock_fall' 'debris_flow'
 'rockfall' 'snow_avalanche' nan 'translational_slide' 'creep' 'lahar'
 'riverbank_collapse']
mudslide  ->  1480
landslide  ->  4752
complex  ->  227
other  ->  28
rock_fall  ->  60
debris_flow  ->  111
rockfall  ->  101
snow_avalanche  ->  6
nan  ->  0
translational_slide  ->  2
creep  ->  4
lahar  ->  7
riverbank_collapse  ->  6


In [22]:
analyze(thing, "country", True)

138
['united states' 'indonesia' nan 'canada' 'peru' 'myanmar (burma)'
 'myanmar' 'new zealand' 'fiji' 'india' 'vanuatu' 'malaysia' 'tajikistan'
 'china' 'australia' 'iceland' 'pakistan' 'argentina' 'nepal' 'brazil'
 'uganda' 'kyrgyzstan' 'nicaragua' 'philippines' 'colombia' 'burma'
 'bolivia' 'kenya' 'ecuador' 'united kingdom' 'bangladesh' 'tibet'
 'papua new guinea' 'afghanistan' 'north korea' 'ethiopia' 'chile'
 'nigeria' 'taiwan' 'honduras' 'japan' 'laos' 'madagascar' 'vietnam'
 'trinidad and tobago' 'ghana' 'brunei' 'mexico' 'rwanda' 'sri lanka'
 'malawi' 'thailand' 'scottland' 'bhutan' 'portugal' 'solomons island'
 'norway' 'south africa' 'russia' 'iran' 'ireland' 'tasmania' 'guatemala'
 'azerbaijan' 'costa rica' 'south korea' 'burkina faso' 'venezuela'
 'georgia' 'jordan' 'democratic republic of congo' 'haiti' 'columbia'
 'tajkikistan' 'tanzania' 'angola' 'france' 'phillipines' 'panama'
 'bulgaria' 'slovakia' 'austria' 'albania' 'cuba' 'lebanon' 'turkey'
 'scotland' 'jamaica' 'u

In [23]:
#date to number of days after first accident
thing["date"] = pd.to_datetime(thing['date'])   
mindate = thing['date'].min()
thing['date'] = (thing['date'] - mindate)  / np.timedelta64(1, 'D')

In [24]:
#split dataset
missingdate = thing[0:0]
missingtype = thing[0:0]
missingcountry = thing[0:0]
missingtrigger = thing[0:0]
missingseverity = thing[0:0]
legit = thing[0:0]

for index, row in thing.iterrows():
    nans = 0
    
    if str(row['country']) == 'nan':
        nans += 1
    if str(row['landslide_type']) == 'nan':
        nans += 1
    if str(row['trigger']) == 'nan':
        nans += 1
    if str(row['severity']) == 'nan':
        nans += 1
    if np.isnan(row['date']):
        if(nans >= 1):
            print("yess but no")
        else:
            print("yess")
        nans += 1
        
    if nans > 1:
        continue
    
    elif str(row['country']) == 'nan':
        missingcountry = missingcountry.append(row, ignore_index = True)
    elif str(row['landslide_type']) == 'nan':
        missingtype = missingtype.append(row, ignore_index = True)
    elif str(row['trigger']) == 'nan':
        missingtrigger = missingtrigger.append(row, ignore_index = True)
    elif str(row['severity']) == 'nan':
        missingseverity = missingseverity.append(row, ignore_index = True)
    elif np.isnan(row['date']):
        missingdate = missingdate.append(row, ignore_index = True)
    else:    
        legit = legit.append(row, ignore_index = True)
'''

    date
    country
    type
    trigger
    severity
    no nulls -> (without noise)   
'''

yess but no
yess but no
yess but no
yess but no


'\n\n    date\n    country\n    type\n    trigger\n    severity\n    no nulls -> (without noise)   \n'

In [25]:
missingdate.head()

,date,country,landslide_type,trigger,severity,latitude,longitude


In [26]:
missingtype.head()

,date,country,landslide_type,trigger,severity,latitude,longitude


In [27]:
missingcountry.head()

,date,country,landslide_type,trigger,severity,latitude,longitude
0,9580.000000,NaN,mudslide,rain,medium,52.3545,-127.6980
1,9427.958333,NaN,mudslide,downpour,small,53.3319,-132.4149
2,9556.000000,NaN,other,downpour,medium,25.6172,96.3005
3,8841.000000,NaN,rock_fall,earthquake,medium,-43.5586,170.1794
4,8841.000000,NaN,complex,no_apparent_trigger,large,-43.5611,170.1796


In [28]:
missingseverity.head()

,date,country,landslide_type,trigger,severity,latitude,longitude
0,9207.000000,malaysia,landslide,downpour,NaN,3.7944,113.6308
1,9207.000000,malaysia,landslide,downpour,NaN,3.8355,113.7131
2,9263.958333,burma,landslide,construction,NaN,25.8509,97.4381
3,9207.000000,malaysia,landslide,downpour,NaN,4.0677,113.8310
4,9253.958333,united states,landslide,downpour,NaN,47.4174,-123.1316


In [29]:
missingtrigger.head()

,date,country,landslide_type,trigger,severity,latitude,longitude
0,9232.000000,united states,landslide,NaN,very_large,58.5624,-137.0834
1,9129.000000,canada,landslide,NaN,small,50.2405,-57.5886
2,9157.000000,canada,landslide,NaN,small,50.2405,-57.5886
3,9100.958333,united states,landslide,NaN,medium,43.7495,-109.9673
4,8451.000000,malaysia,landslide,NaN,medium,1.0571,110.6565


In [30]:
columns = ['date', 'latitude', 'longitude']
[columns.append('landslide_type_' + str(x)) for x in thing['landslide_type'].unique()]
[columns.append('trigger_' + str(x)) for x in thing['trigger'].unique()]
[columns.append('severity_' + str(x)) for x in thing['severity'].unique()]
[columns.append('country_' + str(x)) for x in thing['country'].unique()]

len(columns)

176

In [31]:
scaler = MinMaxScaler(feature_range = (-1, 1))

In [32]:
def get_categorical_missing_values(datasetwithnulls, thing):
    attributes = legit.drop([thing], axis = 1)
    
    attributes = pd.get_dummies(attributes)
    attributes = attributes.T.reindex(columns).T.fillna(0)
    labels = legit[thing]
    
    attributes = scaler.fit_transform(attributes)
    
    attributes_train, attributes_test, labels_train, labels_test = train_test_split(attributes, labels, test_size=0.33)
    
    neigh = KNeighborsClassifier(n_neighbors = 7)
    neigh.fit(attributes_train, labels_train)
    
    f1 = f1_score(labels_test, neigh.predict(attributes_test), average='micro')
    
    print(f1)
    
    attributes = datasetwithnulls.drop([thing], axis = 1)
    dummified = pd.get_dummies(attributes)
    dummified = dummified.T.reindex(columns).T.fillna(0)

    scaled = scaler.transform(dummified)

    predictions = neigh.predict(scaled)
    
    return predictions

In [33]:
get_categorical_missing_values(missingseverity, "severity")

0.7777210015329586


array(['medium', 'medium', 'medium', 'medium', 'medium', 'medium',
       'medium', 'medium', 'medium', 'medium', 'medium', 'medium',
       'medium', 'medium', 'medium', 'medium', 'large', 'medium', 'large',
       'small', 'medium', 'medium', 'medium'], dtype=object)

In [51]:
def get_numerical_missing_values(datasetwithnulls, thing):
    attributes = legit.drop([thing], axis = 1)
    
    attributes = pd.get_dummies(attributes)
    attributes = attributes.T.reindex(columns).T.fillna(0)
    labels = legit[thing]
    
    attributes = scaler.fit_transform(attributes)
    
    attributes_train, attributes_test, labels_train, labels_test = train_test_split(attributes, labels, test_size=0.33)
    
    regression = LinearRegression()
    regression.fit(attributes_train, labels_train)
    
    r2 = r2_score(labels_test, regression.predict(attributes_test))
    
    print(r2)
    
    attributes = datasetwithnulls.drop([thing], axis = 1)
    dummified = pd.get_dummies(attributes)
    dummified = dummified.T.reindex(columns).T.fillna(0)
    
    print(dummified)

    scaled = scaler.transform(dummified)

    predictions = regression.predict(scaled)
    
    return predictions

In [54]:
# theoretically should work, but since our dataset is empty it throws an exception
#get_numerical_missing_values(missingdate, "date")

In [53]:
#restore data
missingcountry["country"] = get_categorical_missing_values(missingcountry, "country")
missingtrigger["trigger"] = get_categorical_missing_values(missingtrigger, "trigger")
missingseverity["severity"] = get_categorical_missing_values(missingseverity, "severity")

0.6739908022483393
0.7143587123147676
0.749105774144098


In [37]:
missingcountry.head()

,date,country,landslide_type,trigger,severity,latitude,longitude
0,9580.000000,united states,mudslide,rain,medium,52.3545,-127.6980
1,9427.958333,united states,mudslide,downpour,small,53.3319,-132.4149
2,9556.000000,india,other,downpour,medium,25.6172,96.3005
3,8841.000000,new zealand,rock_fall,earthquake,medium,-43.5586,170.1794
4,8841.000000,brazil,complex,no_apparent_trigger,large,-43.5611,170.1796


In [38]:
missingtrigger.head()

,date,country,landslide_type,trigger,severity,latitude,longitude
0,9232.000000,united states,landslide,downpour,very_large,58.5624,-137.0834
1,9129.000000,canada,landslide,rain,small,50.2405,-57.5886
2,9157.000000,canada,landslide,rain,small,50.2405,-57.5886
3,9100.958333,united states,landslide,downpour,medium,43.7495,-109.9673
4,8451.000000,malaysia,landslide,downpour,medium,1.0571,110.6565


In [44]:
missingseverity.head()

,date,country,landslide_type,trigger,severity,latitude,longitude
0,9207.000000,malaysia,landslide,downpour,medium,3.7944,113.6308
1,9207.000000,malaysia,landslide,downpour,medium,3.8355,113.7131
2,9263.958333,burma,landslide,construction,medium,25.8509,97.4381
3,9207.000000,malaysia,landslide,downpour,medium,4.0677,113.8310
4,9253.958333,united states,landslide,downpour,medium,47.4174,-123.1316


In [47]:
#concat into one better dataset
better_thing = pd.concat([legit, missingcountry, missingseverity, missingtrigger])

In [48]:
better_thing.head()

,date,country,landslide_type,trigger,severity,latitude,longitude
0,8155.000000,united states,mudslide,downpour,medium,41.5585,-73.4020
1,7084.958333,indonesia,landslide,rain,medium,0.1115,113.9171
2,7990.958333,canada,complex,downpour,medium,50.7053,-127.5062
3,7990.958333,canada,mudslide,downpour,medium,50.4335,-127.4846
4,9025.958333,united states,landslide,snowfall_snowmelt,large,61.9780,-143.1680


In [166]:
#run algorithm
def get_train_model(data):
    #types
    attributes = data.drop(['landslide_type', 'severity', 'latitude', 'longitude'], axis = 1)
    types = data["landslide_type"]
    
    dummified = pd.get_dummies(data)
        
    scaled = MinMaxScaler(feature_range = (-1, 1)).fit_transform(dummified)
    
    types_features_train, types_features_test, types_labels_train, types_labels_test = train_test_split(scaled, types, test_size=0.33)
    
    '''
    k_fold = StratifiedKFold(n_splits=5, shuffle=False)
    k_fold.get_n_splits(types_features_train, types_labels_train)
    
    grid = {"C": [ 1.8, 1.9, 2, 2.1, 2.2]}
    grid_search = GridSearchCV(estimator = LinearSVC(max_iter = 2000), cv = k_fold, param_grid = grid)
    grid_search.fit(types_features_train, types_labels_train)
    linear_svm_classifier = grid_search.best_estimator_

    print("Linear SVM; best score:", grid_search.best_score_)
    '''
    
    #typesclassifier = LinearSVC(C = 1.8, max_iter = 1000)
    typesclassifier = LinearSVC(C = 20, max_iter = 1000)
    typesclassifier.fit(types_features_train, types_labels_train)
    
    
    f1 = f1_score(types_labels_test, typesclassifier.predict(types_features_test), average = 'micro')
    matrix = confusion_matrix(types_labels_test, typesclassifier.predict(types_features_test))
    
    print('types f1 ', f1)
    print(matrix)
    
    '''
    typesreg = LinearSVC()
    typesreg.fit(scaled, types)
    
    f1 = f1_score(types_labels_test, typesreg.predict(types_features_test), average = 'micro')
    matrix = confusion_matrix(types_labels_test, typesreg.predict(types_features_test))
    
    
    
    return typesreg, types_features_test, types_labels_test
    '''
    #severity
    severities = data["severity"]
    severities_features_train, severities_features_test, severities_labels_train, severities_labels_test = train_test_split(scaled, severities, test_size=0.33)
    
    severitiesclassifier = LinearSVC(C = 20, max_iter = 1000)
    severitiesclassifier.fit(severities_features_train, severities_labels_train)
    
    
    f1 = f1_score(severities_labels_test, severitiesclassifier.predict(severities_features_test), average = 'micro')
    matrix = confusion_matrix(severities_labels_test, severitiesclassifier.predict(severities_features_test))
    
    print('severities f1 ', f1)
    print(matrix)
    
    return (typesclassifier, severitiesclassifier)

def get_result(modelcluster, attributes):
    pass

In [167]:
#run algorithm with original data
get_train_model(legit)

C:\Users\Gencho\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Gencho\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


types f1  1.0
[[  71    0    0    0    0    0    0    0    0]
 [   0    1    0    0    0    0    0    0    0]
 [   0    0   18    0    0    0    0    0    0]
 [   0    0    0    2    0    0    0    0    0]
 [   0    0    0    0 1405    0    0    0    0]
 [   0    0    0    0    0  427    0    0    0]
 [   0    0    0    0    0    0    1    0    0]
 [   0    0    0    0    0    0    0   31    0]
 [   0    0    0    0    0    0    0    0    1]]
severities f1  1.0
[[ 162    0    0    0]
 [   0 1424    0    0]
 [   0    0  345    0]
 [   0    0    0   26]]


C:\Users\Gencho\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


(LinearSVC(C=20, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 LinearSVC(C=20, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0))

In [168]:
#run algorithm with better data
get_train_model(better_thing)

C:\Users\Gencho\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Gencho\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


types f1  1.0
[[  69    0    0    0    0    0    0    0    0    0    0]
 [   0    2    0    0    0    0    0    0    0    0    0]
 [   0    0   38    0    0    0    0    0    0    0    0]
 [   0    0    0    4    0    0    0    0    0    0    0]
 [   0    0    0    0 1549    0    0    0    0    0    0]
 [   0    0    0    0    0  483    0    0    0    0    0]
 [   0    0    0    0    0    0    5    0    0    0    0]
 [   0    0    0    0    0    0    0    3    0    0    0]
 [   0    0    0    0    0    0    0    0   16    0    0]
 [   0    0    0    0    0    0    0    0    0   31    0]
 [   0    0    0    0    0    0    0    0    0    0    1]]
severities f1  1.0
[[ 144    0    0    0]
 [   0 1594    0    0]
 [   0    0  436    0]
 [   0    0    0   27]]


C:\Users\Gencho\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


(LinearSVC(C=20, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 LinearSVC(C=20, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0))